In [1]:
import sys
sys.path.append('..')
from data_processor import *
from train_test_split_games import *
from true_skill_through_time import *

In [2]:
boxing_instance = BoxingProcessor("../data/boxer_wiki_urls.txt")

In [3]:
boxing_matches_df = boxing_instance.pull_data()

https://en.wikipedia.org/wiki/William_Abelyan
https://en.wikipedia.org/wiki/Arthur_Abraham
https://en.wikipedia.org/wiki/Miguel_Acosta_(boxer)
https://en.wikipedia.org/wiki/Tomasz_Adamek
https://en.wikipedia.org/wiki/Ola_Afolabi
No suitable table found with at least 4 columns.
https://en.wikipedia.org/wiki/Joachim_Alcine
https://en.wikipedia.org/wiki/Devon_Alexander
https://en.wikipedia.org/wiki/Jos%C3%A9_Alfaro_(boxer)
https://en.wikipedia.org/wiki/Muhammad_Ali
https://en.wikipedia.org/wiki/Canelo_%C3%81lvarez
https://en.wikipedia.org/wiki/Elvis_%C3%81lvarez
No suitable table found with at least 4 columns.
https://en.wikipedia.org/wiki/Sa%C3%BAl_%C3%81lvarez
https://en.wikipedia.org/wiki/Lou_Ambers
https://en.wikipedia.org/wiki/Sammy_Angott
https://en.wikipedia.org/wiki/Alfredo_Angulo
https://en.wikipedia.org/wiki/Vito_Antuofermo
https://en.wikipedia.org/wiki/Fred_Apostoli
https://en.wikipedia.org/wiki/Jorge_Arce
https://en.wikipedia.org/wiki/Alexis_Arg%C3%BCello
https://en.wikipedia.

/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../utils.py:48: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  parsed_dates = pd.to_datetime(date_series.str.replace('–', '-').str.replace('[377]', ''), errors='coerce')
/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../utils.py:48: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  parsed_dates = pd.to_datetime(date_series.str.replace('–', '-').str.replace('[377]', ''), errors='coerce')
/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../utils.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_dates = pd.to_d

Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 1
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 17
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 1
Dropping: 0
Dropping: 0
Dropping: 57
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 37
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 103
Dropping: 0
Dropping: 0
Dropping: 1
Dropping: 0
Dropping: 65
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 71
Dropping: 31
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 55
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Dropping: 0
Droppin

In [4]:
boxing_matches_cleaned_df = boxing_instance.process_games_data(boxing_matches_df)

/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../data_processor.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxing_matches_df['Outcome'] = boxing_matches_df['Result'].apply(lambda x: mapper[x])
/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../data_processor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxing_matches_df[c] = boxing_matches_df[c].apply(lambda x: name_changes[x] if x in name_changes else x)
/Users/davidliu/Desktop/Python312/true_th

In [5]:
boxing_matches_cleaned_df.shape

(25805, 3)

In [6]:
boxing_matches_cleaned_df = boxing_matches_cleaned_df.reset_index()
boxing_matches_cleaned_df = boxing_matches_cleaned_df.rename(columns={'index': 'game_index'})

In [20]:
boxing_matches_cleaned_df.to_parquet('../data/boxing_matches_cleaned.parquet')

In [2]:
boxing_matches_cleaned_df = pd.read_parquet('../data/boxing_matches_cleaned.parquet')

In [3]:
boxing_splitter = BoxingSplitter("../data/boxing_matches_cleaned.parquet", "../data/players_ge_40_matches_lst_boxing.json")

In [9]:
boxing_matches_cleaned_df.head()

,game_index,winner,loser,timestamp
0,0,Jake Kilrain,Jack Daley,1879-01-01
1,1,Jake Kilrain,Jem Driscoll,1879-02-01
2,2,Jake Kilrain,Dan Dwyer,1879-03-10
3,3,John L. Sullivan,Jack Curley,1879-03-13
4,4,John L. Sullivan,Johnny Cocky Woods,1879-03-14


In [4]:
boxing_train, boxing_test = boxing_splitter.train_test_split()

100%|██████████| 303/303 [00:00<00:00, 1106.12it/s]


In [5]:
boxing_train.shape, boxing_test.shape

((16947, 5), (4130, 5))

In [6]:
self = TrueSkillThroughTimeApplied(boxing_train)

In [7]:
self.learn_optimal_parameters(game_type='boxing')


gamma: 0.0200, sigma: 0.0200, beta: 0.0200, 
NLE: 6255.5716

gamma: 0.0200, sigma: 0.0200, beta: 0.0200, 
NLE: 6255.5716

gamma: 0.0200, sigma: 0.0200, beta: 0.0200, 
NLE: 6255.5716

gamma: 0.0200, sigma: 0.0200, beta: 0.0200, 
NLE: 6255.5716

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 11375.8005

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 11375.8006

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 11375.8005

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 11375.8005

gamma: 0.0281, sigma: 0.0281, beta: 0.0281, 
NLE: 5975.5792

gamma: 0.0281, sigma: 0.0281, beta: 0.0281, 
NLE: 5975.5792

gamma: 0.0281, sigma: 0.0281, beta: 0.0281, 
NLE: 5975.5792

gamma: 0.0281, sigma: 0.0281, beta: 0.0281, 
NLE: 5975.5792

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 5901.4197

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 5901.4197

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 5901.4197

gamma: 0.1000, sigma: 0.1000, beta: 0.1000, 
NLE: 5901.4197

gamma: 0.1000, sigm

In [8]:
#NLE: 5786.7752
optimal_gamma = 0.0738
optimal_sigma = 0.6004
optimal_beta = 0.2843
self.set_optimal_parameters(gamma = optimal_gamma, sigma = optimal_sigma, beta = optimal_beta)

In [9]:
skill_curves = self.set_skill_curves()

In [10]:
top_boxers_online = ['Muhammad Ali', 'Joe Louis', 'Sugar Ray Robinson', 'Rocky Marciano', 'Floyd Mayweather Jr', 'Manny Pacquiao', \
                     'Jack Dempsey', 'Roberto Durán', 'Henry Armstrong', 'Willie Pep', 'Mike Tyson']

In [11]:
self.plot_player_skills(players = top_boxers_online, width=1500, burnin=0)

alt.LayerChart(...)

In [12]:
self.plot_calibration()

/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../true_skill_through_time.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_means = df.groupby('win_prob_bucket').agg(


alt.LayerChart(...)

In [13]:
self.plot_calibration_oos(oos_data=boxing_test)

/Users/davidliu/Desktop/Python312/true_theta/posts/trueskill/boxing/../true_skill_through_time.py:276: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_means = df.groupby('win_prob_bucket').agg(


alt.LayerChart(...)

In [14]:
calculate_auc(boxing_test, skill_curves, optimal_beta)

np.float64(0.7463703035968661)